In [1]:
import pandas as pd
import numpy as np
import pdb
from docx import Document
from docx.shared import Mm
from collections import defaultdict

In [2]:
MERSummary = pd.read_excel('MERSummaryAll Wk 22.xlsx')
EMPS = pd.read_excel('EMPSWk22.xlsx')
PDTD = pd.read_excel('PlantDowntimeDetail Wk 22.xlsx')

In [3]:
'''fleet desc import'''
fleetdesc = pd.read_json('fleetdesc.json')
fleetdesc.rename(columns={'Unit no':'Unit'},inplace=True)

In [4]:
'''
MER Processing

'''

MERSummary.drop(MERSummary.columns[[0,1]],axis=1,inplace=True)
MERSummary.dropna(inplace=True)
MERSummary.reset_index(drop=True,inplace=True)
MERSummary.columns = MERSummary.iloc[0]
MERSummary.drop(0,inplace=True)
indexNames = MERSummary[(MERSummary['Unit']=='OEM Totals')|(MERSummary['Unit']=='Class Total')|(MERSummary['Unit']=='Report Total')].index
MERSummary.drop(indexNames, inplace = True)

In [5]:
'''
EMPS Processing

'''

EMPS.drop(EMPS.columns[[0]],axis=1,inplace=True)
EMPS.dropna(inplace=True)
EMPS.reset_index(drop=True,inplace=True)
EMPS.columns = EMPS.iloc[0]
EMPS.drop(0,inplace=True)
EMPS.rename(columns = {'Unit\nNumber':'Unit'},inplace=True)
indexNames2 = EMPS[(EMPS['Unit']=='OEM Totals')|(EMPS['Unit']=='Class Total')|(EMPS['Unit']=='Report Total')].index
EMPS.drop(indexNames2,inplace=True)

In [6]:
'''Processing PDTD'''
PDTD.dropna(thresh=3,inplace=True) #masih PR ya
PDTD.reset_index(drop=True,inplace=True)
PDTD.columns = PDTD.iloc[0]
PDTD.drop(0,inplace=True)
PDTD.rename(columns = {'Unit No.' : 'Unit'},inplace=True)
PDTD_complete = pd.merge(PDTD, fleetdesc, on='Unit', how='inner')
PDTD_complete['Period\nEvent\nMaint.\nDuration'] =  PDTD_complete['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))

In [7]:
'''Combine MER & EMPS'''
Combined = pd.merge(MERSummary,EMPS, on='Unit', how='outer')

In [8]:
'''Combine fleet desc & combined'''
All_KPI = pd.merge(Combined,fleetdesc,on='Unit', how='outer')
All_KPI.drop(columns=['Check','Column7','Column8'],inplace=True)
All_KPI.dropna(thresh=24, inplace=True)
#All_KPI[(All_KPI['fleet desc']=='Cat 16M')]

'''round selected column'''
All_KPI['PMD\n(08)'] = All_KPI['PMD\n(08)'].apply(lambda x: round(x,2))
All_KPI['UMD\n(09)'] = All_KPI['UMD\n(09)'].apply(lambda x: round(x,2))
All_KPI['Total\nEngine On\nPeriod'] =  All_KPI['Total\nEngine On\nPeriod'].apply(lambda x: round(x,2))
All_KPI['PDAM\n08020\n09020'] =  All_KPI['PDAM\n08020\n09020'].apply(lambda x: round(x,2))
#All_KPI['Period\nEvent\nMaint.\nDuration'] =  All_KPI['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))

In [9]:
'''Gather all required variables'''
PWT = All_KPI.groupby(['fleet desc'])['PWT\n(00)'].sum() #F
SWT = All_KPI.groupby(['fleet desc'])['SWT\n(01)'].sum() #G
IODOn = All_KPI.groupby(['fleet desc'])['IOD-On\n(04)'].sum() #H
IODOff = All_KPI.groupby(['fleet desc'])['IOD-Off\n(04)'].sum() #I
EODOn = All_KPI.groupby(['fleet desc'])['EOD-On\n(05)'].sum() #J
EODOff = All_KPI.groupby(['fleet desc'])['EOD-Off\n(05)'].sum() #K
PDAM = All_KPI.groupby(['fleet desc'])['PDAM\n08020\n09020'].sum() #T
PMD = All_KPI.groupby(['fleet desc'])['PMD\n(08)'].sum() #L
UMD = All_KPI.groupby(['fleet desc'])['UMD\n(09)'].sum() #M
EngOn = All_KPI.groupby(['fleet desc'])['Total\nEng On\n(SMU Hrs)'].sum() #V
nFail = All_KPI.groupby(['fleet desc'])['Number of\nFailures\nPeriod'].sum() #W
MTBF_period = All_KPI.groupby(['fleet desc'])['MTBF\nPeriod'].sum() #X
MTTRF_period = All_KPI.groupby(['fleet desc'])['MTTR-F\nPeriod'].sum() #Y

In [10]:
'''Do the math'''
MA = ((PWT+SWT+IODOn+IODOff+EODOn+EODOff+PDAM)/(PWT+SWT+IODOn+IODOff+EODOn+EODOff+PDAM+PMD+UMD-PDAM).replace({0:np.nan})).fillna(0)
MTBF =(EngOn/(nFail).replace({0:np.nan})).fillna(EngOn)
MTTR = ((MTTRF_period)/(nFail).replace({0:np.nan})).fillna(nFail*MTBF_period)

In [11]:
'''Combined all KPI'''
Anomali_db = pd.merge(MA.to_frame(),MTBF.to_frame(), on='fleet desc', how= 'outer')
complete_anomali = pd.merge(Anomali_db, MTTR.to_frame(), on='fleet desc', how= 'outer')
complete_anomali.rename(columns = {'0_x':'MA','0_y':'MTBF'},inplace=True)
complete_anomali.columns = ['MA','MTBF','MTTR']
'''round by 2 decimals'''
complete_anomali['MTBF'] = complete_anomali['MTBF'].apply(lambda x: round(x,2))
complete_anomali['MTTR'] = complete_anomali['MTTR'].apply(lambda x: round(x,2))
'''drop zeros'''
complete_KPI = complete_anomali.loc[~(complete_anomali==0).all(axis=1)]
#complete_KPI['MA'] = pd.Series(['{0:,.2f}%'.format(val * 100) for val in complete_KPI['MA']], index = complete_KPI.index)
complete_KPI

,MA,MTBF,MTTR
fleet desc,,,
CAT 18M,0.727591,113.67,23.46
CAT 745,0.940008,62.65,2.43
Cat 16M,0.862023,49.94,6.13
Cat 740,0.926440,51.45,9.67
Cat 777D Coal Truck,0.944391,39.91,3.72
Cat 777D OB Truck,0.923531,102.29,13.58
Cat 777E Coal Truck,0.938386,122.55,8.00
Cat 785C,0.924524,49.42,4.03
Cat 789C,1.000000,0.00,0.00


Processing MTBF

In [12]:
red_MTBF_this_week = complete_KPI['MTBF'][complete_KPI['MTBF']<100].dropna()
MTBF_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MTBF_this_week.index)]
red_MTBF_this_week

fleet desc
CAT 745                    62.65
Cat 16M                    49.94
Cat 740                    51.45
Cat 777D Coal Truck        39.91
Cat 785C                   49.42
Cat 789C                    0.00
Cat D10R/T                 52.98
Cat D8R                    47.82
Compactor                  31.70
Hitachi EX2500             85.47
Hitachi EX2600             81.80
Hitachi EX3600             44.88
Hitachi EX5500             44.70
Hitachi ZX200              66.80
Hitachi ZX350/ZX330        82.32
KOM D85ESS                 46.38
Komatsu PC300              50.46
Komatsu PC400              76.00
Liebherr R984              35.70
Loader                     83.63
Medium Truck               33.02
Mud Truck                  24.00
TJ RENT Komatsu D65P-12    28.30
TJ RENT Komatsu D85ESS     81.00
TJ Rental PC200            91.70
Trailer/Lowboy             42.00
Volvo A40 Thiess           20.80
Water Truck                63.23
Name: MTBF, dtype: float64

In [13]:
'''MTBF Report template'''
MTBF_report = []


for i in red_MTBF_this_week.index:
    comp_gp_issue = []
    
    '''
    filter downtime detail for only failure event 
    '''
    
    na = MTBF_df[(MTBF_df['Is Failure']=='Yes')&(MTBF_df['fleet desc'] ==i)][['Reason','System']].groupby(['Reason'],as_index=False).count().sort_values('System',ascending=False)['Reason']
    count_of_failure = MTBF_df[(MTBF_df['fleet desc'] ==i)&(MTBF_df['Is Failure']=='Yes')].count()['Is Failure']
    '''
    conditional statement for unit that has no failure
    '''
    if count_of_failure == 0:
        comp_gp_issue.append({})
    else:
        for u in na:    
            '''
            Create dataframe for system failure
            '''
            system_failure = MTBF_df[(MTBF_df['Is Failure']=='Yes')][['fleet desc','Reason','System','Part']].groupby(['fleet desc','Reason','System'],as_index=False).count()
            system_failure['MTBF_gp_repetitive'] = system_failure['System']+' '+system_failure['Part'].values.astype(str)+' times'
            
            '''
            Create dataframe for reason failure failure
            '''
            reason_failure = MTBF_df[(MTBF_df['fleet desc']==i) & (MTBF_df['Is Failure']=='Yes')]['Reason'].value_counts().reset_index()
            reason_failure['Reason_repetitive'] = reason_failure['index']+' '+reason_failure['Reason'].values.astype(str)+' times'

            general_comp_failure = []
            for e in reason_failure['index']:
                f = system_failure[(system_failure['fleet desc']==i)&(system_failure['Reason']==e)]['MTBF_gp_repetitive'].tolist()
                general_comp_failure.append(f)
            repetitive_issue = dict(zip(reason_failure['Reason_repetitive'],general_comp_failure))
        comp_gp_issue.append(repetitive_issue)
    '''
    report template per fleet model
    
    '''
    
    structured_MTBF = {
    'Fleet':i,
    'MTBF':red_MTBF_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'Total Hours Engine ON' : All_KPI.groupby('fleet desc').sum()['Total\nEngine On\nPeriod'][i],
    'Count of failures' : count_of_failure,
    'Failures by Comp Gp with repetitive issue' : comp_gp_issue 
    }
    
    MTBF_report.append(structured_MTBF)
    
MTBF_report

[{'Fleet': 'CAT 745',
  'MTBF': 62.65,
  'Total Unit': 16,
  'Total Hours Engine ON': 1269.1,
  'Count of failures': 11,
  'Failures by Comp Gp with repetitive issue': [{'018 - Chassis/Frame/Cabin/Steer Axles 6 times': ['Chassis 1 times',
     'Suspension 5 times'],
    '011 - Engine 3 times': ['Cooling 2 times', 'Intake/Exhaust 1 times'],
    '019 - LV Electrical 1 times': ['Electrical 1 times'],
    '023 - Aircon 1 times': ['Electrical 1 times']}]},
 {'Fleet': 'Cat 16M',
  'MTBF': 49.94,
  'Total Unit': 6,
  'Total Hours Engine ON': 486.2,
  'Count of failures': 7,
  'Failures by Comp Gp with repetitive issue': [{'019 - LV Electrical 3 times': ['Electrical 1 times',
     'Indicating 1 times',
     'Power 1 times'],
    '011 - Engine 1 times': ['Intake/Exhaust 1 times'],
    '028 - Buckets/Bowls/Body 1 times': ['Blade 1 times'],
    '018 - Chassis/Frame/Cabin/Steer Axles 1 times': ['Chassis 1 times'],
    '017 - Hydraulics 1 times': ['Steering Hydraulic Delivery & Cooling 1 times']}]}

Processing MA

In [14]:
red_MA_this_week = complete_KPI['MA'][complete_KPI['MA']<0.9].dropna()
MA_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MA_this_week.index)]
#MA_df= MA_df['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))
# MA_df= MA_df['UMD\n(09)'].apply(lambda x: round(x,2))
# MA_df= MA_df['PMD\n(08)'].apply(lambda x: round(x,2))

MA_df['MA Anomali']= MA_df['Unit']+' '+MA_df['Reported Fault/Job Description']+' ('+MA_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"
red_MA_this_week
#.Series(['{0:,.2f}%'.format(val * 100) for val in complete_KPI['MA']], index = complete_KPI.index)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\2389035930.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MA_df['MA Anomali']= MA_df['Unit']+' '+MA_df['Reported Fault/Job Description']+' ('+MA_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"


fleet desc
CAT 18M                0.727591
Cat 16M                0.862023
Cat D10R/T             0.805913
Cat D8R                0.858339
Crane                  0.833333
Hitachi ZX350/ZX330    0.834052
Komatsu D65P-12        0.714789
Komatsu PC300          0.755011
Liebherr R984          0.854672
Medium Truck           0.773023
TJ Rental PC200        0.874761
Volvo A40 Thiess       0.877604
Water Truck            0.830570
Name: MA, dtype: float64

In [15]:
MA_df

,Unit,OEM\nDescription,Meter\n(Start),Meter\n(End),Resp.\nGroup,Maintenance\nArea,Event Start Time,Event End Time,Total\nEvent\nMaint.\nDuration,Period\nEvent\nMaint.\nDuration,...,No Work\nRequired,Is Failure,unit desc,fleet desc,additional,Check,Column6,Column7,Column8,MA Anomali
0,EX2467,HIT ZX350 5G,12982.7,12982.7,Field,Field Shop Pit Melawan,2022-05-09 07:00:00,2022-05-18 17:30:00,226.5,48.00,...,No,Yes,HIT ZX350H5G,Hitachi ZX350/ZX330,None,None,Small Excavator,1.0,None,EX2467 Line boring bucket (48.0 Hours)
1,RD3443,KOM HD785 WATER TRUCK 7,38036.4,38036.4,Workshop,Field Shop IGS,2022-04-14 09:30:00,2022-05-30 17:30:00,1112,168.00,...,No,Yes,Komatsu HD785-7,Water Truck,None,None,Big Truck,1.0,None,RD3443 Bevel gear housing crack (168.0 Hours)
2,RD3443,KOM HD785 WATER TRUCK 7,38036.4,38036.4,Workshop,Field Shop IGS,2022-04-14 09:30:00,2022-05-30 17:30:00,1112,168.00,...,No,No,Komatsu HD785-7,Water Truck,None,None,Big Truck,1.0,None,RD3443 Inspection follow up check hydraulic an...
4,RD4389,VOL A40 G,12914.0,12914.0,Workshop,Workshop Melawan,2022-05-10 16:25:00,2022-05-25 17:30:00,361.083333,14.58,...,No,Yes,VOL A40G,Volvo A40 Thiess,None,None,Articulate Truck,1.0,None,RD4389 Inspection follow up : Hydraulic oil lo...
5,RD4389,VOL A40 G,12880.0,12880.0,Field,Other,2022-05-06 08:20:00,2022-05-06 09:45:00,1.416667,1.42,...,No,Yes,VOL A40G,Volvo A40 Thiess,None,None,Articulate Truck,1.0,None,RD4389 Brake sticky (1.42 Hours)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1198,VT3131,IVE AD380 6x6 SERVICE TRUCK TRUCK,15973.5,15973.5,Workshop,Workshop TWS,2022-05-05 06:00:00,2022-05-05 08:00:00,2,2.00,...,No,No,IVE AD380 6x6 SERVICE TRUCK,Medium Truck,Service Truck,None,Support,1.0,None,VT3131 Muffler loose (2.0 Hours)
1199,VT3155,IVE AD240 4x4 MANHAUL TRUCK TRUCK,26696.0,26696.0,Electrical Trade,Workshop TWS,2022-05-05 15:00:00,2022-05-05 16:00:00,1,1.00,...,No,Yes,IVE AD240 4x4 MANHAUL TRUCK,Medium Truck,Manhaul,None,Support,1.0,None,VT3155 Transmission error (1.0 Hours)
1200,VT4034,REN KERAX 330 4x4 MANHAUL TRUCK TRUCK,12666.0,12666.0,Field,Field Shop Pit Melawan,2022-05-07 07:00:00,2022-05-08 12:05:00,29.083333,29.08,...,No,Yes,REN KERAK,Medium Truck,Manhaul,None,Support,1.0,None,VT4034 Cabin cyl fail (29.08 Hours)
1201,VT4034,REN KERAX 330 4x4 MANHAUL TRUCK TRUCK,12664.0,12664.0,Field,Field Shop Pit Melawan,2022-05-08 21:00:00,2022-05-08 22:00:00,1,1.00,...,No,No,REN KERAK,Medium Truck,Manhaul,None,Support,1.0,None,"VT4034 Daily inspection, Planned 1 hrs (1.0 Ho..."


In [16]:
'''MA Report Template'''

MA_report = []

    
    
for i in red_MA_this_week.index:
    Scheduled_maintenance = []
    Unschedule_maintenance = []
    
    for x in MA_df[((MA_df.Activity =='08-Planned Maintenance (PMD)')|(MA_df.Activity ==',08-Planned Maintenance (PMD)'))&(MA_df['fleet desc'] ==i)&(MA_df['Period\nEvent\nMaint.\nDuration']>10.0)]['MA Anomali']:
        Scheduled_maintenance.append(x)
    
    for x in MA_df[((MA_df.Activity =='09-Unplanned Maintenance (UMD)')|(MA_df.Activity ==',09-Unplanned Maintenance (UMD)'))&((MA_df['fleet desc'] ==i)&(MA_df['Period\nEvent\nMaint.\nDuration']>10.0))]['MA Anomali']:
        Unschedule_maintenance.append(x)
    
    
    structured_MA = {
        
    
    'Fleet':i,
    'MA':red_MA_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'UMD total hours':round(All_KPI.groupby('fleet desc').sum()['UMD\n(09)'][i],2),
    'PMD total hours':All_KPI.groupby('fleet desc').sum()['PMD\n(08)'][i],
    'PDAM total hours':All_KPI.groupby('fleet desc').sum()['PDAM\n08020\n09020'][i],
    'Scheduled maintenance' : Scheduled_maintenance,
    'Unscheduled maintenance' : Unschedule_maintenance
    
}
    MA_report.append(structured_MA)

MA_report

[{'Fleet': 'CAT 18M',
  'MA': 0.7275909963779692,
  'Total Unit': 6,
  'UMD total hours': 205.17,
  'PMD total hours': 69.42,
  'PDAM total hours': 0.0,
  'Scheduled maintenance': ['GR1396 PM 2000HRS Service (Duration 9 hrs) and backlog repair, Planned 48 hrs (35.67 Hours)',
   'GR1396 Backlog repair (018) (35.67 Hours)',
   'GR1396 Backlog repair 026  (35.67 Hours)',
   'GR1396 Backlog repair 017  (35.67 Hours)',
   'GR1396 Backlog repair 013  (35.67 Hours)',
   'GR1396 Electrical inspection (35.67 Hours)',
   'GR1398 Backlog repair (G) (103.5 Hours)'],
  'Unscheduled maintenance': ['GR1397 Inspection follow up : Final drive POS 3 Malfunction (38.67 Hours)',
   'GR1398 Over run : Continue completed related part after artic hitch replacement (103.5 Hours)',
   'GR1398 Over run : Continue electrical inspection (103.5 Hours)',
   'GR1398 Over run : Continue backlog repair  (103.5 Hours)',
   'GR1398 Over run : Continue backlog repair  (103.5 Hours)',
   'GR1399 Circle stuck  (51.42 Hours

Processing MTTR

In [17]:
red_MTTR_this_week = complete_KPI['MTTR'][complete_KPI['MTTR']>6].dropna()
MTTR_df = PDTD_complete[PDTD_complete['fleet desc'].isin(red_MTTR_this_week.index)]
MTTR_df['Period\nEvent\nMaint.\nDuration'] = MTTR_df['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))
MTTR_df['MTTR Anomali']= MTTR_df['Unit']+' '+MTTR_df['Description of Repair']+' ('+MTTR_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"
red_MTTR_this_week

C:\Users\User\AppData\Local\Temp\ipykernel_14832\4244589648.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MTTR_df['Period\nEvent\nMaint.\nDuration'] = MTTR_df['Period\nEvent\nMaint.\nDuration'].apply(lambda x: round(x,2))
C:\Users\User\AppData\Local\Temp\ipykernel_14832\4244589648.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MTTR_df['MTTR Anomali']= MTTR_df['Unit']+' '+MTTR_df['Description of Repair']+' ('+MTTR_df['Period\nEvent\nMaint.\nDuration'].values.astype(str)+" Hours)"


fleet desc
CAT 18M                23.46
Cat 16M                 6.13
Cat 740                 9.67
Cat 777D OB Truck      13.58
Cat 777E Coal Truck     8.00
Cat D10R/T              7.09
Hitachi EX2500          7.78
Hitachi ZX350/ZX330    28.76
KSB Rent Pump          25.75
Komatsu PC300          14.03
Loader                 10.55
Medium Truck            8.03
Mud Truck               7.67
Pumps                  16.35
TJ Rental PC200        18.54
Water Truck            21.53
Name: MTTR, dtype: float64

In [18]:
'''MTTR Report Template'''

MTTR_report = []
 
    
for i in red_MTTR_this_week.index:
    Breakdown_detail = []
    
    for x in MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)&(MTTR_df['Period\nEvent\nMaint.\nDuration'])]['MTTR Anomali']:
        Breakdown_detail.append(x)
        
    structured_MTTR = {
    'Fleet':i,
    'MTTR':red_MTTR_this_week[i],
    'Total Unit':All_KPI.groupby('fleet desc').count()['Unit'][i],
    'Hours Maintenance with Failure is Yes' : round(MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)].sum()['Period\nEvent\nMaint.\nDuration'],2),
    'Count of Failures' : MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)].count()['Is Failure'],
    'Detail Breakdown with Failure is Yes' : Breakdown_detail
    
}
    MTTR_report.append(structured_MTTR)

C:\Users\User\AppData\Local\Temp\ipykernel_14832\1114159306.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  'Hours Maintenance with Failure is Yes' : round(MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)].sum()['Period\nEvent\nMaint.\nDuration'],2),
C:\Users\User\AppData\Local\Temp\ipykernel_14832\1114159306.py:16: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  'Hours Maintenance with Failure is Yes' : round(MTTR_df[(MTTR_df['Is Failure'] =='Yes')&(MTTR_df['fleet desc'] ==i)].sum()['Period\nEvent\nMaint.\nDuration'],2),
C:\Users\User\AppData\Local\Temp\ipykernel_14832\1114159306.py:16: FutureWarning: Dropping of nuisance columns

In [19]:
MTTR_report

[{'Fleet': 'CAT 18M',
  'MTTR': 23.46,
  'Total Unit': 6,
  'Hours Maintenance with Failure is Yes': 91.84,
  'Count of Failures': 4,
  'Detail Breakdown with Failure is Yes': ['GR1396 Repair FRONT worklamp connector (0.67 Hours)',
   'GR1398 Re-oring discharge hose implement pump, top up hydraulic oil 65 litre (1.17 Hours)',
   'GR1399 08/05/2022 N12: Check Circle Drive Stuck, Remove Cover Circle Drive, Inspect Gear and Shaft Broken., 09/05/2022 N12: Completed pinion circle drive, Install lower cover circle drive, Install circle drive box to unit, pull out bolt  mounting  circle shoes  broken  3EA, Install circle include strip wear and adjust clearance pinion gear (51.42 Hours)',
   'GR1405 05/05/2022 N12: FOUND : Circle drive box broken. Remove bolt mounting circle drive, Remove pinion gear, Flushing oil circle drive, 06/05/2022 D12: Fabrication shoe circle loose 1 ea, 06/05/2022 N12: Replace Circle Drive BOX, Pre load Disc pack, Fill Up Oil Circle, Install Circle Shoe 2ea, Replace C

Write down to the word document

In [20]:
'''Create word document using word'''
Anomali_report_full = Document()

section = Anomali_report_full.sections[0]
section.page_height = Mm(297)
section.page_width = Mm(210)
section.left_margin = Mm(25.4)
section.right_margin = Mm(25.4)
section.top_margin = Mm(25.4)
section.bottom_margin = Mm(25.4)
section.header_distance = Mm(12.7)
section.footer_distance = Mm(12.7)


'''Mechanical Availability'''


'''
formatting MA table
'''

table_MA = Anomali_report_full.add_table(rows=1,cols=3)
table_MA.style = 'Table Grid'
hdr_cells_MA = table_MA.rows[0].cells
hdr_cells_MA[0].text = 'Fleet/Model'
hdr_cells_MA[1].text = 'MA this week'
hdr_cells_MA[2].text = 'Comments on Non-Compliance'

for i in MA_report:
    
    TOU_MA = i.get('Total Unit')
    PMD_MA = i.get('PMD total hours')
    UMD_MA = i.get('UMD total hours')
    PDAM_MA = i.get('PDAM total hours')
    
    row_cells_MA = table_MA.add_row().cells
    row_cells_MA[0].text = i.get('Fleet')
    row_cells_MA[1].text = str(i.get('MA'))
    
    sch = i.get('Scheduled maintenance')
    Unsch = i.get('Unscheduled maintenance')
    
    Scheduled_maintenance = '\n\n'.join([c for c in sch[1:]])
    Unscheduled_maintenance = '\n\n'.join([c for c in Unsch[1:]])
    
    non_compliance = f'Number of Units = {TOU_MA} \nPMD Total = {PMD_MA} \nUMD Total= {UMD_MA} \nPDAM Total hours= {PDAM_MA}\n Scheduled maintenance :\n{Scheduled_maintenance}\nUnscheduled maintenance :\n{Unscheduled_maintenance}'

    row_cells_MA[2].text = non_compliance
    
'''
Mean Time Between Failure
'''

'''
formatting MTBF table 
'''

table_MTBF = Anomali_report_full.add_table(rows=1, cols=3)
table_MTBF.style = 'Table Grid'

hdr_cells_MTBF = table_MTBF.rows[0].cells
hdr_cells_MTBF[0].text = 'Fleet/Model '
hdr_cells_MTBF[1].text = 'MTBF this week'
hdr_cells_MTBF[2].text = 'Comments on Non-Compliance'

for d in MTBF_report:
    
    
    
    TOU = d.get('Total Unit')
    THEON = d.get('Total Hours Engine ON')
    COF = d.get('Count of failures')
    
    
    
    
    row_cells = table_MTBF.add_row().cells
    row_cells[0].text = d.get('Fleet')
    row_cells[1].text = str(d.get('MTBF'))
    
    gp_components = []
    
    
    for system in d.get('Failures by Comp Gp with repetitive issue'):
            for key, value in system.items():
                s = ", ".join(value)
                ter = f'{key}: {s}'
                gp_components.append(ter)
            
            
    f_list = '\n'.join([d for d in gp_components[1:]])
    gp_comp_list = f'Number of Units = {TOU} \nTotal Hours Engine On = {THEON} \n Count of failures = {COF} \n Failures by Comp Gp with repetitive issue : \n {f_list}'
    
    row_cells[2].text = gp_comp_list

'''
Mean Time to Repair Failure
'''

'''
formatting MTTRF table 
'''
table_MTTR = Anomali_report_full.add_table(rows=1, cols=3)
table_MTTR.style = 'Table Grid'

hdr_cells_MTTR = table_MTTR.rows[0].cells
hdr_cells_MTTR[0].text = 'Fleet/Model '
hdr_cells_MTTR[1].text = 'MTTR this week'
hdr_cells_MTTR[2].text = 'Comments on Non-Compliance'

for i in MTTR_report:
    
    TOU_MTTRF = i.get('MTTR')
    Hours_Fail_MTTRF = i.get('Hours Maintenance with Failure is Yes')
    COF_MTTRF = i.get('Count of Failures')
    DBF_MTTRF = i.get('Detail Breakdown with Failure is Yes')
    detail_breakdown_MTTRF = '\n'.join([c for c in DBF_MTTRF[1:]])
        
    comment_MTTR = f'Number of Units = {TOU} \nHours maintenance with is Failure Yes ={Hours_Fail_MTTRF} \nCount of failures/Is Failure Yes = {COF_MTTRF} \n\n\n Detail breakdown with failure is yes:\n{detail_breakdown_MTTRF}'
    
    row_cells = table_MTTR.add_row().cells
    row_cells[0].text = i.get('Fleet')
    row_cells[1].text = str(i.get('MTTR'))
    row_cells[2].text = comment_MTTR

Anomali_report_full.save('Anomali_trial1.docx')
del Anomali_report_full

In [21]:
# Will later be fixed :
# -Still choosing how the product will be used : software package or web

# Major :
# 1. Applied different standard of MTBF for different unit type (now all unit type has same standard >90)
# 2. Filter for only extreme hours.
# 3. MTBF value for AW unit that has low engine on hours but has no failure event.

# Minor : 
# 1. Change MA format to %
# 2. Bullet list
# 3. Column width 
# 4. Column color
# 5. Page break